In [1]:
import pandas as pd
import plotly.express as px


In [2]:
# Cargar dataset
df = pd.read_csv("/workspaces/Mapa/Anexo EA4_Tourist_Travel_Europe.csv")

# Agrupar por país y calcular métricas
agg_df = df.groupby("Country_Visited").agg({
    "Total_Travel_Cost": "mean",
    "Travel_Duration_Days": "mean",
    "Number_of_Companions": "mean",
    "Accommodation_Type": lambda x: x.value_counts().index[0],  # más frecuente
    "Main_Purpose": lambda x: x.value_counts().index[0],        # más frecuente
    "Season_of_Visit": lambda x: x.value_counts().index[0]      # más frecuente
}).reset_index()

# Crear mapa coroplético refinado
fig = px.choropleth(
    agg_df,
    locations="Country_Visited",
    locationmode="country names",
    color="Total_Travel_Cost",
    hover_name="Country_Visited",
    hover_data={
        "Travel_Duration_Days": True,
        "Number_of_Companions": True,
        "Accommodation_Type": True,
        "Main_Purpose": True,
        "Season_of_Visit": True,
        "Total_Travel_Cost": True
    },
    color_continuous_scale="Viridis",
    title="Mapa Coroplético Interactivo: Variables Significativas por País"
)

fig.show()

/tmp/ipykernel_6073/1360050068.py:15: DeprecationWarning: The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.
  fig = px.choropleth(


### Ocio vs Negocios por Pais

In [3]:
# Filtrar solo viajes de ocio y negocios
filtered_df = df[df["Main_Purpose"].isin(["Leisure", "Business"])]

# Agrupar por país y propósito
agg_df = filtered_df.groupby(["Country_Visited", "Main_Purpose"]).agg({
    "Total_Travel_Cost": "mean",
    "Travel_Duration_Days": "mean"
}).reset_index()

# Crear mapa coroplético con contraste ocio vs negocios
fig = px.choropleth(
    agg_df,
    locations="Country_Visited",
    locationmode="country names",
    color="Total_Travel_Cost",
    facet_col="Main_Purpose",  # Paneles separados: ocio vs negocios
    hover_name="Country_Visited",
    hover_data={
        "Travel_Duration_Days": True,
        "Total_Travel_Cost": True
    },
    color_continuous_scale="Viridis",
    title="Mapa Coroplético: Comparación Ocio vs Negocios por País"
)

fig.show()


/tmp/ipykernel_6073/4028366099.py:11: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



### Tipo de alojamiento mas frecuente

In [4]:
# Calcular tipo de alojamiento más frecuente por país
agg_df = df.groupby("Country_Visited")["Accommodation_Type"] \
           .agg(lambda x: x.value_counts().index[0]) \
           .reset_index()

# Crear mapa coroplético categórico
fig = px.choropleth(
    agg_df,
    locations="Country_Visited",
    locationmode="country names",
    color="Accommodation_Type",  # variable categórica
    hover_name="Country_Visited",
    title="Mapa Coroplético: Tipo de Alojamiento Más Frecuente por País",
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.show()


/tmp/ipykernel_6073/554897776.py:7: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



In [7]:
# Agrupar por ciudad y calcular promedios
agg_df = df.groupby("City_Visited").agg({
    "Travel_Duration_Days": "mean",
    "Total_Travel_Cost": "mean"
}).reset_index()

# Diccionario de coordenadas aproximadas de las ciudades
city_coords = {
    "Lisbon": (38.7169, -9.1390),
    "Porto": (41.1496, -8.6110),
    "Faro": (37.0194, -7.9304),
    "Madrid": (40.4168, -3.7038),
    "Barcelona": (41.3851, 2.1734),
    "Seville": (37.3891, -5.9845),
    "Rome": (41.9028, 12.4964),
    "Venice": (45.4408, 12.3155),
    "Milan": (45.4642, 9.1900),
    "Berlin": (52.5200, 13.4050),
    "Hamburg": (53.5511, 9.9937),
    "Munich": (48.1351, 11.5820),
    "London": (51.5074, -0.1278),
    "Manchester": (53.4808, -2.2426),
    "Edinburgh": (55.9533, -3.1883),
    "Paris": (48.8566, 2.3522),
    "Lyon": (45.7640, 4.8357),
    "Nice": (43.7102, 7.2620),
    "Amsterdam": (52.3676, 4.9041),
    "Rotterdam": (51.9225, 4.4792),
    "Utrecht": (52.0907, 5.1214),
    "Vienna": (48.2100, 16.3738),
    "Salzburg": (47.8095, 13.0550),
    "Innsbruck": (47.2692, 11.4041),
    "Athens": (37.9838, 23.7275),
    "Thessaloniki": (40.6401, 22.9444),
    "Santorini": (36.3932, 25.4615),
    "Lucerne": (47.0502, 8.3093),
    "Zurich": (47.3769, 8.5417),
    "Geneva": (46.2044, 6.1432)
}

# Añadir coordenadas al dataframe
agg_df["lat"] = agg_df["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[0])
agg_df["lon"] = agg_df["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[1])

# Crear mapa interactivo con burbujas
fig = px.scatter_geo(
    agg_df,
    lat="lat",
    lon="lon",
    text="City_Visited",
    size="Travel_Duration_Days",  # tamaño según duración promedio
    color="Total_Travel_Cost",    # color según costo promedio
    hover_name="City_Visited",
    hover_data={"Travel_Duration_Days": True, "Total_Travel_Cost": True},
    color_continuous_scale="Viridis",
    title="Mapa Interactivo: Ciudades vs Duración y Costo Promedio"
)

fig.update_geos(scope="europe", showland=True, landcolor="lightgray")
fig.show()


In [8]:
# Calcular medio de transporte más frecuente por país
agg_df = df.groupby("Country_Visited")["Mode_of_Travel"] \
           .agg(lambda x: x.value_counts().index[0]) \
           .reset_index()

# Crear mapa coroplético categórico
fig = px.choropleth(
    agg_df,
    locations="Country_Visited",
    locationmode="country names",
    color="Mode_of_Travel",  # variable categórica
    hover_name="Country_Visited",
    title="Mapa Coroplético: Medio de Transporte Más Frecuente por País",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.show()


/tmp/ipykernel_6073/162397213.py:7: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



In [10]:
# Agrupar por país y medio de transporte
agg_df = df.groupby(["Country_Visited", "Mode_of_Travel"]).size().reset_index(name="Count")

# Crear mapa coroplético con facet por medio de transporte
fig = px.choropleth(
    agg_df,
    locations="Country_Visited",
    locationmode="country names",
    color="Count",  # cantidad de viajes
    facet_col="Mode_of_Travel",  # paneles separados por transporte
    hover_name="Country_Visited",
    hover_data={"Count": True, "Mode_of_Travel": True},
    color_continuous_scale="Viridis",
    title="Mapa Coroplético: Cantidad de Viajes por Medio de Transporte y País"
)

fig.show()


/tmp/ipykernel_6073/365125329.py:5: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.



In [13]:
# Filtrar solo vuelos
flights_df = df[df["Mode_of_Travel"] == "Flight"]

# Contar vuelos por ciudad
city_flights = flights_df["City_Visited"].value_counts().reset_index()
city_flights.columns = ["City_Visited", "Flight_Count"]

# Diccionario de coordenadas aproximadas
city_coords = {
    "Lisbon": (38.7169, -9.1390),
    "Porto": (41.1496, -8.6110),
    "Faro": (37.0194, -7.9304),
    "Madrid": (40.4168, -3.7038),
    "Barcelona": (41.3851, 2.1734),
    "Seville": (37.3891, -5.9845),
    "Rome": (41.9028, 12.4964),
    "Venice": (45.4408, 12.3155),
    "Milan": (45.4642, 9.1900),
    "Berlin": (52.5200, 13.4050),
    "Hamburg": (53.5511, 9.9937),
    "Munich": (48.1351, 11.5820),
    "London": (51.5074, -0.1278),
    "Manchester": (53.4808, -2.2426),
    "Edinburgh": (55.9533, -3.1883),
    "Paris": (48.8566, 2.3522),
    "Lyon": (45.7640, 4.8357),
    "Nice": (43.7102, 7.2620),
    "Amsterdam": (52.3676, 4.9041),
    "Rotterdam": (51.9225, 4.4792),
    "Utrecht": (52.0907, 5.1214),
    "Vienna": (48.2100, 16.3738),
    "Salzburg": (47.8095, 13.0550),
    "Innsbruck": (47.2692, 11.4041),
    "Athens": (37.9838, 23.7275),
    "Thessaloniki": (40.6401, 22.9444),
    "Santorini": (36.3932, 25.4615),
    "Lucerne": (47.0502, 8.3093),
    "Zurich": (47.3769, 8.5417),
    "Geneva": (46.2044, 6.1432)
}

# Añadir coordenadas
city_flights["lat"] = city_flights["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[0])
city_flights["lon"] = city_flights["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[1])

# Crear mapa interactivo
fig = px.scatter_geo(
    city_flights,
    lat="lat",
    lon="lon",
    text="City_Visited",
    size="Flight_Count",       # tamaño según cantidad de vuelos
    color="Flight_Count",      # color según cantidad de vuelos
    hover_name="City_Visited",
    hover_data={"Flight_Count": True},
    color_continuous_scale="Plasma",
    title="Mapa Interactivo: Cantidad de Vuelos que Llegan por Ciudad"
)

fig.update_geos(scope="europe", showland=True, landcolor="lightgray")
fig.show()


In [15]:
# Agrupar por ciudad y tipo de alojamiento
agg_df = df.groupby(["City_Visited", "Accommodation_Type"]).size().reset_index(name="Count")

# Seleccionar el alojamiento más frecuente por ciudad
dominant_accommodation = agg_df.loc[agg_df.groupby("City_Visited")["Count"].idxmax()].reset_index(drop=True)

# Diccionario de coordenadas aproximadas
city_coords = {
    "Lisbon": (38.7169, -9.1390),
    "Porto": (41.1496, -8.6110),
    "Faro": (37.0194, -7.9304),
    "Madrid": (40.4168, -3.7038),
    "Barcelona": (41.3851, 2.1734),
    "Seville": (37.3891, -5.9845),
    "Rome": (41.9028, 12.4964),
    "Venice": (45.4408, 12.3155),
    "Milan": (45.4642, 9.1900),
    "Berlin": (52.5200, 13.4050),
    "Hamburg": (53.5511, 9.9937),
    "Munich": (48.1351, 11.5820),
    "London": (51.5074, -0.1278),
    "Manchester": (53.4808, -2.2426),
    "Edinburgh": (55.9533, -3.1883),
    "Paris": (48.8566, 2.3522),
    "Lyon": (45.7640, 4.8357),
    "Nice": (43.7102, 7.2620),
    "Amsterdam": (52.3676, 4.9041),
    "Rotterdam": (51.9225, 4.4792),
    "Utrecht": (52.0907, 5.1214),
    "Vienna": (48.2100, 16.3738),
    "Salzburg": (47.8095, 13.0550),
    "Innsbruck": (47.2692, 11.4041),
    "Athens": (37.9838, 23.7275),
    "Thessaloniki": (40.6401, 22.9444),
    "Santorini": (36.3932, 25.4615),
    "Lucerne": (47.0502, 8.3093),
    "Zurich": (47.3769, 8.5417),
    "Geneva": (46.2044, 6.1432)
}

# Añadir coordenadas
dominant_accommodation["lat"] = dominant_accommodation["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[0])
dominant_accommodation["lon"] = dominant_accommodation["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[1])

# Crear mapa interactivo
fig = px.scatter_geo(
    dominant_accommodation,
    lat="lat",
    lon="lon",
    text="City_Visited",
    size="Count",                 # tamaño según cantidad de viajes
    color="Accommodation_Type",   # color según tipo de alojamiento
    hover_name="City_Visited",
    hover_data={"Accommodation_Type": True, "Count": True},
    title="Mapa Interactivo: Cantidad del Alojamiento Más Frecuente por Ciudad",
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_geos(scope="europe", showland=True, landcolor="lightgray")
fig.show()


In [16]:
# Calcular total de personas por ciudad (viajero principal + acompañantes)
df["Total_Personas"] = df["Number_of_Companions"] + 1
city_visits = df.groupby("City_Visited")["Total_Personas"].sum().reset_index()

# Diccionario de coordenadas aproximadas
city_coords = {
    "Lisbon": (38.7169, -9.1390),
    "Porto": (41.1496, -8.6110),
    "Faro": (37.0194, -7.9304),
    "Madrid": (40.4168, -3.7038),
    "Barcelona": (41.3851, 2.1734),
    "Seville": (37.3891, -5.9845),
    "Rome": (41.9028, 12.4964),
    "Venice": (45.4408, 12.3155),
    "Milan": (45.4642, 9.1900),
    "Berlin": (52.5200, 13.4050),
    "Hamburg": (53.5511, 9.9937),
    "Munich": (48.1351, 11.5820),
    "London": (51.5074, -0.1278),
    "Manchester": (53.4808, -2.2426),
    "Edinburgh": (55.9533, -3.1883),
    "Paris": (48.8566, 2.3522),
    "Lyon": (45.7640, 4.8357),
    "Nice": (43.7102, 7.2620),
    "Amsterdam": (52.3676, 4.9041),
    "Rotterdam": (51.9225, 4.4792),
    "Utrecht": (52.0907, 5.1214),
    "Vienna": (48.2100, 16.3738),
    "Salzburg": (47.8095, 13.0550),
    "Innsbruck": (47.2692, 11.4041),
    "Athens": (37.9838, 23.7275),
    "Thessaloniki": (40.6401, 22.9444),
    "Santorini": (36.3932, 25.4615),
    "Lucerne": (47.0502, 8.3093),
    "Zurich": (47.3769, 8.5417),
    "Geneva": (46.2044, 6.1432)
}

# Añadir coordenadas
city_visits["lat"] = city_visits["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[0])
city_visits["lon"] = city_visits["City_Visited"].map(lambda x: city_coords.get(x, (None,None))[1])

# Crear mapa interactivo
fig = px.scatter_geo(
    city_visits,
    lat="lat",
    lon="lon",
    text="City_Visited",
    size="Total_Personas",       # tamaño según cantidad de personas
    color="Total_Personas",      # color según cantidad de personas
    hover_name="City_Visited",
    hover_data={"Total_Personas": True},
    color_continuous_scale="Viridis",
    title="Mapa Interactivo: Cantidad de Personas que Visitan Cada Ciudad"
)

fig.update_geos(scope="europe", showland=True, landcolor="lightgray")
fig.show()
